In [1]:
TOKEN = '182QTV1K4ZK4ILFP'

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [113]:
def get_snp_tickers():
    
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table', {'class': 'wikitable sortable'})

    sp500_tickers = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip()
        sp500_tickers.append(ticker)

    return sp500_tickers

In [117]:
def get_df(data):
    data = data.json()
    
    try:
        df = pd.DataFrame(data['feed'])
        df = df[[
        'title',
        'url',
        'time_published',
        'summary',
        'source',
        'category_within_source',
        'source_domain',
        'overall_sentiment_score',
        'overall_sentiment_label',
        'ticker_sentiment']]
        df = pd.concat([df.explode('ticker_sentiment').drop(['ticker_sentiment'], axis=1),
               df.explode('ticker_sentiment')['ticker_sentiment'].apply(pd.Series)],
              axis=1)
        df['time_published'] = pd.to_datetime(df['time_published'], format='%Y%m%dT%H%M%S')
        return df 
    except:
        return pd.DataFrame()

In [ ]:
tickers = get_snp_tickers()
df = pd.DataFrame()

chunks = [tickers[i:i + 5] for i in range(0, len(tickers), 5)]

for chunk in chunks:
    for ticker in chunk:
        print(ticker)
        url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={TOKEN}'
        r = requests.get(url)
        df = pd.concat([df, get_df(r)], ignore_index=True)
    time.sleep(70)

In [ ]:
df.ticker.value_counts()[:20]

In [121]:
df.to_csv('alpha_vantage_news.csv', index=False)

In [122]:
df.count()

title                      112467
url                        112467
time_published             112467
summary                    112467
source                     112467
category_within_source     112467
source_domain              112467
overall_sentiment_score    112467
overall_sentiment_label    112467
ticker                     112467
relevance_score            112467
ticker_sentiment_score     112467
ticker_sentiment_label     112467
dtype: int64